In [ ]:
import requests                  # [handles the http interactions](http://docs.python-requests.org/en/master/) 
from bs4 import BeautifulSoup    # beautiful soup handles the html to text conversion and more
import re                        # regular expressions are necessary for finding the crumb (more on crumbs later)
from datetime import timedelta,datetime    # string to datetime object conversion
from time import mktime          # mktime transforms datetime objects to unix timestamps

########################################################
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance
    
    parameters: stock - short-handle identifier of the company 
    
    returns a tuple of header, crumb and cookie
    """
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)


########################################################
def convert_to_unix(date):
    """
    converts date to unix timestamp
    
    parameters: date - in format (dd-mm-yyyy)
    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


########################################################
def load_csv_data(stock, interval='1d', day_begin=(datetime.now()-timedelta(days=14)).strftime('%d-%m-%Y'), day_end=datetime.now().strftime('%d-%m-%Y')):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company
        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data
        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)
        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')[:-1]

In [4]:
########################################################
csv = load_csv_data('0700.HK')
for c in csv:
    print c

Date,Open,High,Low,Close,Adj Close,Volume
2019-01-14,329.600006,329.600006,320.200012,321.799988,321.799988,18977242
2019-01-15,325.000000,330.799988,323.200012,330.000000,330.000000,16953757
2019-01-16,330.000000,331.000000,325.200012,331.000000,331.000000,16392202
2019-01-17,335.000000,338.000000,330.600006,331.200012,331.200012,23148086
2019-01-18,335.000000,337.799988,333.799988,337.000000,337.000000,15656802
2019-01-21,338.000000,344.399994,336.200012,340.000000,340.000000,19062984
2019-01-22,342.000000,342.000000,330.799988,335.799988,335.799988,21039177
2019-01-23,330.600006,335.600006,330.600006,333.200012,333.200012,16410980
2019-01-24,332.200012,333.000000,326.799988,330.200012,330.200012,17113453
2019-01-25,335.000000,343.799988,333.399994,343.799988,343.799988,31534946
